In [1]:
import re

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,metrics,callbacks

from typing import List
from json import load,dump

from threading import Thread

In [2]:
with open("../data/example_train.json","r") as build_file:
    build_config = eval(build_file.read())
    
with open("../data/example_code.py","r") as code_file:
    code = code_file.read()

In [3]:
# build_config

In [4]:
# if len(re.findall("""'REQUIRED'""",build_config.__str__())):
#     pass

In [5]:
# build_config.__str__()

In [6]:
def execute_code(_code:str)->None:
    exec(_code)
    globals().update(locals())

In [7]:
class TfGui(keras.callbacks.Callback):
    batch = None 
    epoch = 0
    
    batch_size=8
    batches = 1
    epochs = 1
    
    trainer = None
    
    def __repr__(self,):
        return f"""TfGui(
    batch={self.batch},
    logs={self.logs},
    batch_size={self.batch_size},
    epochs={self.epochs},
)
"""
    
    @property
    def status(self,):
        return {
            "epochs":self.epochs,
            "batchs":self.batches
        }
    
    def __init__(self,trainer):
        self.trainer = trainer
    
    def on_train_begin(self,epoch,logs=None):
        pass
    
    def on_batch_end(self,batch,logs=None):
        self.batch = batch
        self._epoch['log'] = {
            "batch":batch,
            "output":logs
        }
        
    def on_epoch_begin(self,epoch,logs=None):
        self.epoch = epoch
        self._epoch = {
            "epoch":epoch,
            "log":{
                "batch":0,
                "output":None
            },
            "train":{
                "epochs":self.epochs,
                "batches":self.batches
            }
        }
        
        self.trainer.update_log(
            log_type = "epoch",
            log=self._epoch
        )

In [15]:
class Summary(object):
    def __init__(self):
        pass
    
    
    def get(self,build_config:dict,code:str)->None:
        
        charset = 'a-zA-Z0-9 .\(\)\{\{\[\] \n=\-_\+,\'\"'
        imports, = re.findall("""#import[a-zA-Z\n .,_\-]+#end-import""",code)
        execute_code(imports)
        execute_code(build_config['train_config']['dataset']['value'])
        
        for level in build_config['levels']:
            for layer in level:
                _code = re.findall(f"""{layer} =[{charset}]+#end-{layer}""",code)
                if len(_code):
                    execute_code(_code[0])

        if build_config['train_config']['optimizer']:
            execute_code(build_config['train_config']['optimizer']['value'])

        if build_config['train_config']['callbacks']:
            for callback in build_config['train_config']['callbacks']:
                execute_code(callback['value'])    
        
        if build_config['train_config']['loss']:
            execute_code(build_config['train_config']['loss']['value'])
            
        comp = build_config['train_config']['compile']['id']
        model = build_config['train_config']['model']['id']
        execute_code(re.findall(f"""{model}.compile[{charset}]+#end-{comp}""",code)[0])
        
        summary = []
        def print_fn(word,*arg,**kwargs):
            summary.append(word,)
            
        model = globals()[model]
        model.summary(line_length = 512, print_fn=print_fn)

        return summary[::2]
    
summary = Summary()

In [16]:
print ("\n".join(summary.get(build_config,code)))

Model: "model_2"
Layer (type)                                                                                                                                                                                                                          Output Shape                                                                                                                                                                                                 Param #                                                                      
input_3 (InputLayer)                                                                                                                                                                                                                  [(None, 224, 224, 3)]                                                                                                                                                                                        0                                  